In [ ]:
import os

if "notebooks" in os.getcwd():
    %cd ..
from pathlib import Path

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns
from src.utils.defines import INTERIM_DATA_DIR ,AUGMENTED_DATA_DIR, PROCESSED_DATA_DIR, RAW_DATA_DIR
import matplotlib.patches as mpatches

In [340]:
######################################################################################################
# REFORMAT OUTPUT FROM UNLABELED - task b
task_b_labeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_b.csv"))
unlabeled_task_b_GAB = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_GAB.csv"))
# Set a confidence threshold for pseudo-labeling
confidence_threshold = 0.80
test = np.array(task_b_labeled)

# Get the most confident predictions and their indices
most_confident_indices = np.where(test.max(axis=1) > confidence_threshold)[0]
most_confident_preds = pd.DataFrame(test.argmax(axis=1)[most_confident_indices])
most_confident_preds.columns = ['target_b']

text_task_b = unlabeled_task_b_GAB.iloc[most_confident_indices].reset_index(drop=True)

final_task_b = pd.concat([text_task_b, most_confident_preds],axis=1)
final_task_b

,text,target_b
0,usa are bitches of the zog gang she needs to g...,0
1,I probably wouldn't brag about that because us...,3
2,Now I feel like we should just rib that whore....,0
3,Not everyone can be brave and talk shit online...,2
4,"And if you don't want to hear racist crap, SEL...",2
...,...,...
3225,Moochelle-ms don't plan ahead. When 85y.o. mar...,2
3226,"That, and I don't even know how we'd wiggle mo...",2
3227,We have had msm contact us to ask what it is a...,2
3228,THAT'S obama's WHORE MUDDER GIMMMY DA FIST DADDY,2


In [341]:
unique_values, counts = np.unique(most_confident_preds, return_counts=True)

for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0: 348
1: 791
2: 1803
3: 288


In [342]:
dummy_values = np.random.rand(len(final_task_b))
df = final_task_b.copy()
df = df.assign(target_a=pd.Series(dummy_values))
df = df.assign(target_c=pd.Series(dummy_values))
df = df.assign(rewire_id=pd.Series(dummy_values))
df = df.reindex(columns=['rewire_id', 'text', 'target_a', 'target_b', 'target_c'])
df.to_csv("task_b_GAB_aug.csv", index=False)